Para descargar mas datos necesitamos Selenium, porque Beautifulsoup no va a funcionar. Motivo la busqueda por fechas en el  sitio esta basado en Javascript.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import html5lib

import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.common.keys import Keys

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import time
from functions import *

%matplotlib inline
import matplotlib
import seaborn as sns

In [3]:
page = 'https://www.camara.cl/trabajamos/sala_votaciones.aspx'
start_date = '01/01/2016'
end_date = '08/07/2017'
browser = webdriver.Firefox()


browser.get(page)
browser.find_element_by_xpath("//select[@id='ctl00_mainPlaceHolder_ddlBuscarPor']/option[@value='fecha']").click()
time.sleep(10)

inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_txtFecha1')
inputElement.clear()
time.sleep(1)
inputElement.click()
inputElement.send_keys(start_date)
inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_txtFecha2')
inputElement.clear()
time.sleep(1)
inputElement.click()
inputElement.send_keys(end_date)

inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_btnBuscar')
inputElement.click()
time.sleep(60)




In [ ]:


votingDataPage = []
failedPageIndex = []
counter = 0
finished = False
while not finished:
    # Get Data of current page
    votingDataPage, failedPageIndex, soup, failed = readPage(counter, browser, votingDataPage, failedPageIndex)
    print('Done Parsing Page Index ', counter)
    counter = counter + 1
    
    # Confirm next link and tha we are not finished
    nextLink, finished = getNextLink(browser)

    # Get next link
    # Take care if we landed in a dummy page
    getNextPage(browser, nextLink, 90, failed)
    

# combine data frames
votingData = pd.concat(votingDataPage)


Starting to Read Page Index 0


In [ ]:
votingData.vote.unique()

In [ ]:
voteValues = {'A Favor' : 1.0,
              'En Contra' : -1.0,
              'Abstención' : -2.0,
              'Dispensados' : -3.0,
              'Pareo' : -4.0}

votingData['vote2'] = [voteValues[x] for x in votingData.vote]

votingData.sample(5)


In [ ]:
#votingData['date1'] = votingData.date.str.replace('.', '') 
votingData['date1'] = pd.to_datetime(votingData['date'], format='%d de %b de %Y %H:%M')
votingData['dateDoc'] = votingData['date1'].astype(str) + ' - ' + votingData['Documento']
votingData.head()

In [ ]:
votingData = votingData.set_index(pd.DatetimeIndex(votingData['date1']))
votingData.groupby('dateDoc')['Person'].nunique().plot(ylim=(0, 120))

In [ ]:
votingData.to_csv('../output/votingData_TestMoreTime-20170716.csv')